In [1]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn
import pandas as pd
import sys
import scipy
from IPython.core.display import HTML
HTML("<style>.container { width:85% !important; }</style>")

In [2]:
paths = ['/home/j/jt/jt306/PycharmProjects','/home/j/jt/jt306/PycharmProjects/FeatureSelectionPython278','/home/j/jt/jt306/PycharmProjects/FeatureSelectionPython278/bahsic']
for item in paths:
    if item not in sys.path:
        sys.path.insert(0,item)
# print(sys.path)
import FeatureSelectionPython278 as pj

In [6]:
from CollateResults import collate_single_dataset, collate_all_datasets

from SingleFoldSlice import make_directory, Experiment_Setting

In [7]:
all_data =[]
all_attributes = []

def add_attributes_and_scores(all_attributes,all_data,s):
    scores = collate_all_datasets(s)
    attributes = [s.name,s.classifier,s.lupimethod,s.featsel,s.take_top_t,s.topk,s.percent_of_priv]
    all_attributes.append(attributes)
    all_data.append(scores)
    
# Add results for baseline setting to dataframe
s = Experiment_Setting(foldnum='all', topk='all', dataset='tech', datasetnum='all', skfseed=1,
                                 take_top_t='top', lupimethod='nolufe', featsel='nofeatsel', classifier='baseline')
print(s.name)
add_attributes_and_scores(all_attributes,all_data,s)

# Add results for featselector settings to dataframe
for featsel in ['rfe','anova','chi2','mi','bahsic']:
    s = Experiment_Setting(foldnum='all', topk=300, dataset='tech', datasetnum='all', skfseed=1,
                                 take_top_t='top', lupimethod='nolufe', featsel=featsel, classifier='featselector')
    add_attributes_and_scores(all_attributes,all_data,s)

# Add results for lufe settings to dataframe
for lufe in ['dp', 'svmplus','dsvm']:
    for featsel in ['rfe','anova','chi2','mi','bahsic']:
        s = Experiment_Setting(foldnum='all', topk=300, dataset='tech', datasetnum='all', skfseed=1,
                                     take_top_t='top', lupimethod=lufe, featsel=featsel, classifier='lufe')
        add_attributes_and_scores(all_attributes,all_data,s)

# Add results for different amounts of privileged data
for top_bottom in ['top','bottom']:
    for percentofpriv in [10,25,50,75]:
        s = Experiment_Setting(foldnum='all', topk=300, dataset='tech', datasetnum='all', skfseed=1,
                                     take_top_t=top_bottom, lupimethod='svmplus', featsel='rfe', classifier='lufe', percent_of_priv=percentofpriv)
        add_attributes_and_scores(all_attributes,all_data,s)

for top_bottom in ['top']:
    for percentofpriv in [100]:
        s = Experiment_Setting(foldnum='all', topk=300, dataset='tech', datasetnum='all', skfseed=1,
                                     take_top_t=top_bottom, lupimethod='svmplus', featsel='rfe', classifier='lufe', percent_of_priv=percentofpriv)
        add_attributes_and_scores(all_attributes,all_data,s)
        
all_attributes = np.array(all_attributes)
all_data = np.mean(np.array(all_data),axis=2)

print(all_data.shape)
print(all_attributes.shape)
all_attributes= np.hstack([all_attributes,all_data])
df = pd.DataFrame(all_attributes)
columns = ['Name','Classifier','Lupi Method','Feature Selection','take_top_t','# Selected','% Privileged Used']+([str(item) for item in (range(1,296))])
df.columns = columns                
df[[str(item) for item in (range(1,296))]] = df[[str(item) for item in (range(1,296))]].apply(pd.to_numeric)
                                                
df['mean']= (df[[str(item) for item in (range(1,296))]].mean(axis=1))
df[['Name','Classifier','Lupi Method','Feature Selection','take_top_t','# Selected','% Privileged Used','mean']]#.sort_values(by='mean')

baseline-nolufe-nofeatsel-allselected-top100priv-0.1


AssertionError: 

In [10]:
df[:22][['Classifier','Lupi Method','Feature Selection','% Privileged Used','mean']]

,Classifier,Lupi Method,Feature Selection,% Privileged Used,mean
0,baseline,nolufe,nofeatsel,100,0.812259
1,featselector,nolufe,rfe,100,0.826242
2,featselector,nolufe,anova,100,0.838855
3,featselector,nolufe,chi2,100,0.839455
4,featselector,nolufe,mi,100,0.850012
5,featselector,nolufe,bahsic,100,0.837866
6,lufe,dp,rfe,100,0.837400
7,lufe,dp,anova,100,0.846812
8,lufe,dp,chi2,100,0.847159
9,lufe,dp,mi,100,0.859850


# Subsets of results

## 1) Using subset of priv

In [20]:
# df[[df['% Privileged Used'].isin([10,25])]]#,['Classifier','Lupi Method','Feature Selection','# Selected','% Privileged Used','mean']]
df[21:30][['Classifier','Lupi Method','Feature Selection','take_top_t','% Privileged Used','mean']]

,Classifier,Lupi Method,Feature Selection,take_top_t,% Privileged Used,mean
21,lufe,svmplus,rfe,top,10,0.839671
22,lufe,svmplus,rfe,top,25,0.841513
23,lufe,svmplus,rfe,top,50,0.842330
24,lufe,svmplus,rfe,top,75,0.842706
25,lufe,svmplus,rfe,bottom,10,0.821282
26,lufe,svmplus,rfe,bottom,25,0.842929
27,lufe,svmplus,rfe,bottom,50,0.846997
28,lufe,svmplus,rfe,bottom,75,0.847592
29,lufe,svmplus,rfe,top,100,0.843448


In [23]:
def get_all_indices(number=295): 
    return [str(item) for item in (range(1,number+1))]

s = Experiment_Setting(foldnum='all', topk='all', dataset='tech', datasetnum='all', skfseed=1,
                                 take_top_t='top', lupimethod='nolufe', featsel='nofeatsel', classifier='baseline')
def get_results_from_setting(s):
    return(df.loc[df['Name'] == s.name,get_all_indices()].apply(pd.to_numeric))


get_results_from_setting(s)
# df.loc[df['column_name'] == some_value]


,1,2,3,4,5,6,7,8,9,10,...,286,287,288,289,290,291,292,293,294,295
0,0.855383,0.852564,0.905925,0.717399,0.594542,0.615613,0.645655,0.687738,0.739608,0.663333,...,0.90243,0.773407,0.871429,0.828554,0.922153,0.735238,0.826272,0.754338,0.797602,0.742892


In [24]:
s1 = Experiment_Setting(foldnum='all', topk=300, dataset='tech', datasetnum='all', skfseed=1,
                                     take_top_t='top', lupimethod='svmplus', featsel='rfe', classifier='lufe')
s2 = Experiment_Setting(foldnum='all', topk=300, dataset='tech', datasetnum='all', skfseed=1,
                                     take_top_t='top', lupimethod='dp', featsel='rfe', classifier='lufe')

In [29]:
def compare_two_settings(s1, s2):    
#     return df.loc[[index1,index2],get_all_indices()].diff().loc[[2]]
#     return df.loc[[df['Name'] == s1.name or s2.name],get_all_indices()].diff().loc[[2]]
    return df[df['Name'].isin([s1.name,s2.name])][get_all_indices()].diff().loc[[2]]
    
#     return frame[get_all_indices()].diff()
    
compare_two_settings(s1,s2)

KeyError: 'None of [[2]] are in the [index]'

In [27]:
# plt.bar(range(295),compare_two_settings(1,2))
compare_two_settings(s1,s2).plot(kind='bar',color='black',legend='False',xticks=True)
plt.gca().legend_.remove()
plt.show()

TypeError: object of type 'bool' has no len()

# Comparing LUFe settings to their corresponding SVM setting

In [ ]:
for 